# 최종 정리

## 요구사항

이 것을 왜 하냐? 계속 구글 스프레스 시트로 반복 작업을 하고 싶지 않았기 때문이다.

- Ajax 로 특정 시군구 긁어오기
- 정렬, 복사, 붙여넣기 등
- 스프레트 시트에 지역-탭 별로 나누기

이 정보의 최종 활용자는? 50대 컴퓨터에 익숙치 않은 어른. 그가 이 프로그램을 조작하진 않는다. 그리고 난 그에게 엑셀 파일만 주면 된다.

몇 개의 필드를 숨기거나 하는 작업을 해도 된다. 그렇지만, 자료를 업데이트할 때마다 매번 자료의 형태를 가공해야하는 작업이 싫다.

즉, 최종적으로 나는

1. 내가 원하는 지역 경기도, 인천, 서울의 모든 아파트 정보를
    - 각 페이지에
    - 함수 한 번으로 가져오고
    - 저장한 다음에
    - 이것을 아래의 형태로 가공해서 각 부천,시흥시,광명시,구로구,양천구,부평구 페이지에 저장한다.
    - 양식을 붙여넣을 수 있기 때문에.

2. 최신 정보를 갱신할 때는.
    - 기존의 정보의 명칭과 새로 받아온 것의 명칭을 비교해서
    - 정보가 틀릴 때는 최신 것으로 업데이트하고
    - 정보가 없을 때는 새로 추가하며
    - 정보가 같을 때는 아무일도 하지 않는다.

그렇게 되면 나는 이미 가공된 자료의 필드 몇개를 숨기거나 양식을 붙여넣는 수준으로 작업을 편하게 할 수 있다.

연번|명칭|법정동|난방|관리사무실|팩스|준공일|방식|연차|동수|방광|요일|시간|금액|선입|문어|매수|1차|2차|비고
--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--
ㅇ|

## 피드백

비판과 재비판을 함 해보려고 한다.

**원하는 지역의 아파트 코드만 가져온다**

1. 필요한 건 오직 특정 시군구 정보 뿐이다.
 - 아니다. 특정 시군구 법정코드와 아파트 코드가 필요하다.
2. 특정 시군구 아파트 코드만 필요하다
 - 가능하려면 2.x 메가짜리 파일에서 특정 동을 검색해서 그부분만 잘라서 xml 로 만든 뒤에 매번 csv 임포트 해줘야 한다.

즉, sql 에 현재 가진 모든 법정코드와 주소를 테이블로 넣어두고 아파트 정보가 디테일하게 담긴 테이블에 정보를 넣어두는 것이 좋다.

**아파트 코드만 얻어오면 아파트 정보를 얻을 수 있다.**

- 그러나 읽어온 아파트 정보는 처리가 필요하다.
- 게다가 영문으로 된 정보가 한글로 무엇인지 번역될 필요가 있다.

```
http://www.k-apt.go.kr/kaptinfo/getKaptInfo_detail.do?kapt_code=A10027255
```

In [ ]:
  import ( "io/ioutil";"net/http";"strings";"fmt";"bytes" )

    res, err := http.Get("http://www.k-apt.go.kr/kaptinfo/getKaptInfo_detail.do?kapt_code=A10027255")
    defer res.Body.Close()
    body, err := ioutil.ReadAll(res.Body); buf := bytes.NewBuffer(body); json := buf.String(); fmt.Println(json[:300])
  	

{"resultMap_match":{"KAPT_CODE":"A10027255","TOWN_CODE":20306348},"resultMap_kapt":{"CODE_HEAT":"지역난방","SUBWAY_STATION":"-","SUBWAY_LINE":"7호선, 인천선","KAPT_PE1AREA":0,"KAPT_PE2AREA":0,"KAPT_PE3AREA":0,"KAPT_PEAREA":0,"KAPT_PE4AREA":0,"DISPOSAL_TYPE":"분무식","KAPT_PE5AREA":0,"KAP
301
<nil>


# 구현 

## 모든 행정동 코드 정보를 테이블에 넣기

### 파일을 읽어서 존재하는 정보만 배열로 준비하기

In [4]:
import ("encoding/csv"; "fmt"; "os"; "github.com/kniren/gota/dataframe";)

// 접속 생성 및  닫기 지연 걸기
 csvFile, err := os.Open("./법정동코드_전체자료.txt")
 defer csvFile.Close()

// 일단 리더생성 후 쉼표 옵션 기호를 탭으로 설정
 reader := csv.NewReader(csvFile)
 reader.Comma = '\t' // Use tab-delimited instead of comma <---- here!
 reader.FieldsPerRecord = -1

// 모두 읽어제껴버림
 arrayData, err := reader.ReadAll()

// 헤더를 제외한 데이터만 배열에 담기
var bjdArrData [][]string
ind := 0
for index, row := range arrayData{
  
    if (index == 0){
        // bjdArrData = append(filteredArrData, row[:2])
        // index가 0 이면 다시 for 룹 시작으로 가서 index 1 부터 처리
        // 즉 헤더 빼고 데이터처리만 하라는 말
        continue
    }else if(row[2]=="존재"){
        bjdArrData = append(bjdArrData, row[:2])
    }
}

fmt.Println(bjdArrData[:4])

[[1100000000 서울특별시] [1111000000 서울특별시 종로구] [1111010100 서울특별시 종로구 청운동] [1111010200 서울특별시 종로구 신교동]]
168
<nil>


### 테이블 생성 후 배열 자료를 추가하기 

생성

In [16]:
import (
	"database/sql"
	"log"
	"os"
    "fmt"
    
    pg "github.com/lib/pq"
)

    _ = pg.Efatal

const (
    DB_USER     = "gopher"
    DB_PASSWORD = "1111"
    DB_NAME     = "gopher" // postgres create DB named created user's
    DB_HOST        = "db"
)
 
dbinfo := fmt.Sprintf("user=%s password=%s dbname=%s host=%s sslmode=disable",
    DB_USER, DB_PASSWORD, DB_NAME, DB_HOST)

db, err := sql.Open("postgres", dbinfo)

if err != nil {
    log.Println(err)
}
defer db.Close()

createtable:= `CREATE TABLE 법정동 (
                법정동코드 bigint NOT NULL DEFAULT 0,
                법정동명 varchar(30)  NOT NULL DEFAULT ''
              )`

// 쿼리 날릴 준비를 하고 실행한다.
stmt, err1 := db.Prepare(createtable)

defer stmt.Close()


_, err = stmt.Exec()
if err != nil {
    fmt.Println(err.Error())
}

벌크 임포트(대량 가져오기) https://godoc.org/github.com/lib/pq#hdr-Bulk_imports

prepared statement http://go-database-sql.org/prepared.html


In [17]:
// 접속
db, err := sql.Open("postgres", dbinfo)
defer db.Close()


txn, err := db.Begin()
if err != nil {
	fmt.Println(err)
}
// func CopyIn(table string, columns ...string) string
stmt, err := txn.Prepare(pg.CopyIn("법정동", "법정동코드", "법정동명"))
if err != nil {
	fmt.Println(err)
}

for _, bjd := range bjdArrData {
//     _, err = stmt.Exec(int64(bjd[0]), string(bjd[1]))
        _, err = stmt.Exec(bjd[0], bjd[1])
	if err != nil {
		fmt.Println(err)
	}
}

_, err = stmt.Exec()
if err != nil {
	fmt.Println(err)
}

err = stmt.Close()
if err != nil {
	fmt.Println(err)
}

err = txn.Commit()
if err != nil {
	fmt.Println(err)
}

## 특정 키워드를 입력하면 해당하는 모든 법정동 코드 가져오는 함수 만들기

### 함수 설계 (mock)

In [42]:
func getBjd(areaName string) []int {
    
    법정동코드결과 := []int{}
    
    // areaName Like 쿼리 결과 를 법정동 코드결과 변수에 append 로 넣는다.
    
    법정동코드결과 = append(법정동코드결과, 1001, 1002, 1003)
    
    return 법정동코드결과
}

fmt.Println(getBjd("지역키워드"))

[1001 1002 1003]
17
<nil>


### 쿼리 설계

In [46]:
// db 접속 포인터를 얻고
db, err := sql.Open("postgres", dbinfo)

if err != nil { log.Println(err)}
defer db.Close()

// Query the database.
rows, err := db.Query(`
    SELECT 법정동코드
    FROM 법정동
    WHERE 법정동명 LIKE $1
    `, "%서울%")

if err != nil { log.Println(err)}
defer rows.Close()
    
fmt.Println("질의 날리기")

법정동코드결과 := []string{}

for rows.Next() {
    
    코드하나 := 0

if err := rows.Scan(&코드하나); err != nil {
log.Println(err)}
 
   법정동코드결과 =  append(법정동코드결과,코드하나)
// fmt.Printf("%.2d, \n", 법정동코드)
}
    
fmt.Println("출력하기", 법정동코드결과[:4])
    

28:58: cannot use 코드하나 (variable of type int) as string value in argument to append


### 지역 키워드에 따라 법정동 코드를 가져오는 함수 완성

In [45]:
func getBjd(areaName string) []int {
    
    법정동코드결과 := []int{}
    
    // areaName Like 쿼리 결과 를 법정동 코드결과 변수에 append 로 넣는다.
    db, err := sql.Open("postgres", dbinfo)
    if err != nil { log.Println(err)}
    defer db.Close()
    
    // Query the database.
    rows, err := db.Query(`SELECT 법정동코드 FROM 법정동 WHERE 법정동명 LIKE $1`,
                      "%"+ areaName +"%")
    for rows.Next() {    
        코드하나 := 0
        if err := rows.Scan(&코드하나); err != nil {log.Println(err)}
        법정동코드결과 =  append(법정동코드결과,코드하나)
    }
    
    return 법정동코드결과
}

fmt.Println(getBjd("구례"))

[4155034021 4471038027 4481033026 4673000000 4673025000 4673025021 4673025022 4673025023 4673025024 4673025025 4673025026 4673025027 4673025028 4673025029 4673025030 4673031000 4673031021 4673031022 4673031023 4673031024 4673032000 4673032021 4673032022 4673032023 4673032024 4673032025 4673032026 4673032027 4673032028 4673032029 4673033000 4673033021 4673033022 4673033023 4673033024 4673033025 4673033026 4673033027 4673033028 4673033029 4673033030 4673034000 4673034021 4673034022 4673034023 4673034024 4673034025 4673034026 4673035000 4673035021 4673035022 4673035023 4673035024 4673035025 4673035026 4673035027 4673035028 4673036000 4673036021 4673036022 4673036023 4673036024 4673036025 4673036026 4673036027 4673037000 4673037021 4673037022 4673037023 4673037024 4673037025 4673037026 4673037027 4673037028 4673037029 4673037030 4673037031 4673037032 4673037033 4673037034 4673037035 4679034035 4715036027]
915
<nil>


## 아파트 기본 정보 담을 테이블 생성하기

### xml 정보 확인

https://tutorialedge.net/golang/parsing-xml-with-golang/

https://www.thepolyglotdeveloper.com/2017/03/parse-xml-data-in-a-golang-application/

response body 만 xml 로 바꿔서 xml 파싱한다.

### 헤더 테이블 매칭 관계 파악

https://www.data.go.kr/subMain.jsp#/L3B1YnIvcG90L215cC9Jcm9zTXlQYWdlL29wZW5EZXZHdWlkZVBhZ2UkQF4wMTIkQF5wdWJsaWNEYXRhRGV0YWlsUGs9dWRkaTphMTJjNmU3Ni04OTgyLTRlM2UtYWZlMi05NDQxMmM3ZDBhNzkkQF5tYWluRmxhZz10cnVl

### xml 구조 생성

https://tutorialedge.net/golang/parsing-xml-with-golang/

In [ ]:
import (
	"encoding/xml"
	"fmt"
	"io/ioutil"
	"os"
)

type ResMsg struct {
	XMLName xml.Name `xml:"result"`
    결과코드 xml.Name `xml:"resultCode"`
    결과메시지 xml.Name `xml:"resultMag"`
	Items   []Item   `xml:"Items"`
}

type Item struct {
    Items   []Item   `xml:"Items"`
    결과코드 string `xml:"resultCode"`
    결과메시지 string `xml:"resultMag"`
    아파트코드	string `xml:"kaptCode"`
    아파트명	string `xml:"kaptName"`
    법정동주소	string `xml:"kaptAddr"`
    분양형태	string `xml:"codeSaleNm"`
    난방방식	stringe `xml:"codeHeatNm"`
    건축물대장상 연면적	string `xml:"kaptTarea"`
    동수	string `xml:"kaptDongCnt"`
    세대수	string `xml:"kaptdaCnt"`
    시공사	string `xml:"kaptBcompany"`
    시행사	stringe `xml:"kaptAcompany"`
    관리사무소연락처	string `xml:"kaptTel"`
    관리사무소팩스	string `xml:"kaptFax"`
    홈페이지주소	stringe `xml:"kaptUrl"`
    단지분류	stringe `xml:"codeAptNm"`
    도로명주소	string `xml:"doroJuso"`
    호수	string `xml:"hoCnt"`
    관리방식	string `xml:"codeMgrNm"`
    복도유형	xml.Name `xml:"codeHallNm"`
    사용승인일	xml.Name `xml:"kaptUsedate"`
    관리비부과면적	xml.Name `xml:"kaptMarea"`
    전용면적별 세대현황	xml.Name `xml:"kaptMparea_60"`
    전용면적별 세대현황	xml.Name `xml:"kaptMparea_85"`
    전용면적별 세대현황	xml.Name `xml:"kaptMparea_135"`
    전용면적별 세대현황	xml.Name `xml:"kaptMparea_136"`
}
